In [1]:
import pickle
from dataclasses import asdict
from pathlib import Path

import pyvista as pv

from cardiac_electrophysiology.ulac import preprocessing, segmentation

## Load Data

In [2]:
patient_id = "01"
infile_name = f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk"
feature_tags = segmentation.FeatureTags(MV=0, LAA=1, LIPV=2, LSPV=3, RIPV=4, RSPV=5)

vtk_mesh = pv.read(infile_name)
triangular_mesh = preprocessing.convert_unstructured_to_polydata_mesh(vtk_mesh)

## Extract All Boundaries, Convert to Ordered Paths

In [3]:
boundaries = segmentation.extract_boundaries(triangular_mesh, feature_tags)
boundary_paths = segmentation.construct_boundary_paths(boundaries)

## Construct Connection Paths

In [4]:
roof_connection_paths = segmentation.construct_roof_connection_paths(
    triangular_mesh, boundary_paths
)
diagonal_mv_connection_paths = segmentation.construct_diagonal_mv_connection_paths(
    triangular_mesh, boundary_paths
)
pv_inner_outer_connection_paths = segmentation.construct_pv_inner_outer_connection_paths(
    triangular_mesh, boundary_paths, roof_connection_paths, diagonal_mv_connection_paths
)

## Get Boundary Markers

In [5]:
pv_markers = segmentation.get_pv_markers(pv_inner_outer_connection_paths)
laa_markers = segmentation.get_laa_markers(diagonal_mv_connection_paths)
mv_markers = segmentation.get_mv_markers(diagonal_mv_connection_paths)

## Save

In [6]:
save_file = Path("segmentation.pkl")
objects_to_save = [boundary_paths, pv_markers, laa_markers, mv_markers]

with save_file.open("wb") as f:
    pickle.dump(objects_to_save, f)

## Visualize

In [7]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(triangular_mesh, color="lightgray", show_edges=True, edge_opacity=0.3)
for boundary in asdict(boundary_paths).values():
    boundary_points = triangular_mesh.points[boundary]
    plotter.add_points(boundary_points, color="blue", point_size=5, render_points_as_spheres=True)
for roof_connection in asdict(roof_connection_paths).values():
    roof_connection_points = triangular_mesh.points[roof_connection]
    plotter.add_points(
        roof_connection_points, color="yellow", point_size=5, render_points_as_spheres=True
    )
for diagonal_mv_connection in asdict(diagonal_mv_connection_paths).values():
    diagonal_mv_connection_points = triangular_mesh.points[diagonal_mv_connection]
    plotter.add_points(
        diagonal_mv_connection_points, color="green", point_size=5, render_points_as_spheres=True
    )
for pv_inner_outer_connection in asdict(pv_inner_outer_connection_paths).values():
    anterior_posterior = triangular_mesh.points[pv_inner_outer_connection["anterior_to_posterior"]]
    septal_lateral = triangular_mesh.points[pv_inner_outer_connection["septal_to_lateral"]]
    diagonal = triangular_mesh.points[pv_inner_outer_connection["diagonal"]]
    plotter.add_points(
        anterior_posterior, color="yellow", point_size=5, render_points_as_spheres=True
    )
    plotter.add_points(septal_lateral, color="yellow", point_size=5, render_points_as_spheres=True)
    plotter.add_points(diagonal, color="green", point_size=5, render_points_as_spheres=True)
for marker in asdict(pv_markers).values():
    anterior_posterior = triangular_mesh.points[marker["anterior_posterior"]]
    septal_lateral = triangular_mesh.points[marker["septal_lateral"]]
    diagonal = triangular_mesh.points[marker["diagonal"]]
    plotter.add_points(
        anterior_posterior, color="red", point_size=10, render_points_as_spheres=True
    )
    plotter.add_points(septal_lateral, color="red", point_size=10, render_points_as_spheres=True)
    plotter.add_points(diagonal, color="red", point_size=10, render_points_as_spheres=True)
plotter.show()

Widget(value='<iframe src="http://localhost:46859/index.html?ui=P_0x7f1181b4d2b0_0&reconnect=auto" class="pyvi…